# Görüntü Sınıflandırması (Image classification)

Bu çalışma dosyası, çiçek görüntülerinin nasıl sınıflandırılacağını gösterir. Bir `tf.keras.Sequential` modeli kullanarak bir görüntü sınıflandırıcı oluşturur ve `tf.keras.utils.image_dataset_from_directory` kullanarak verileri yükler. Aşağıdaki kavramlarla pratik deneyim kazanacaksınız:

* Bir veri kümesini diskten verimli bir şekilde yükleme.
* Fazla takmayı belirleme ve veri artırma ve bırakma da dahil olmak üzere bunu hafifletmek için teknikleri uygulama.

Bu eğitici, temel bir makine öğrenimi iş akışını takip eder:

1. Verileri inceleyin ve anlayın
2. Bir girdi ardışık düzeni oluşturun
3. Modeli oluşturun
4. Modeli eğitin
5. Modeli test edin
6. Modeli geliştirin ve işlemi tekrarlayın

### TensorFlow Kütüphanesini ve Diğer Kütüphaneleri Ekleme

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Veri Kümesini İndirin ve Keşfedin

Bu çalışma dosyası, yaklaşık 3.700 çiçek fotoğrafından oluşan bir veri kümesi kullanır. Veri kümesi, sınıf başına bir tane olmak üzere beş alt dizin içerir:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

İndirdikten sonra, artık mevcut veri kümesinin bir kopyasına sahip olmalısınız. Toplam 3.670 resim var:

In [ ]:
data_dir

In [ ]:
!ls /root/.keras/datasets/flower_photos

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

İşte bazı güller (roses):

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

Ve bazı laleler(tulips):

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

# Keras yardımcı fonksiyonunu kullanarak verileri yükleyin

`tf.keras.utils.image_dataset_from_directory` yardımcı fonksiyonunu kullanarak bu görüntüleri diskten yükleyelim. 
Bu, sizi diskteki bir görüntü dizininden sadece birkaç satır kodla bir `tf.data.Dataset`e götürecektir. İsterseniz, [Resimleri yükleme ve önişleme](../load_data/images.ipynb) çalışma dosyasını ziyaret ederek kendi veri yükleme kodunuzu sıfırdan da yazabilirsiniz.

## Bir veri kümesi oluşturun

Yükleyici için bazı parametreleri tanımlayın:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Modelinizi geliştirirken bir doğrulama kümesi kullanmak iyi bir uygulamadır. Görsellerin %80'ini eğitim için ve %20'sini doğrulama için kullanalım.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Bu veri kümelerinde sınıf adlarını `class_names` özniteliğinde bulabilirsiniz. Bunlar alfabetik sırayla dizin adlarına karşılık gelir.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Verileri görselleştirin

İşte eğitim veri kümesinden ilk dokuz görüntü:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Bu veri kümelerini kullanarak bir modeli hemen `Model.fit`e geçirerek eğiteceksiniz. İsterseniz, veri kümesini manuel olarak yineleyebilir ve toplu görüntü alabilirsiniz:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

`image_batch`, "(32, 180, 180, 3)" şeklinin bir tensörüdür. 
Bu, `180x180x3` şeklindeki 32 görüntüden oluşan bir toplu işlemdir (son boyut, RGB renk kanallarını ifade eder). `label_batch`, `(32,)` şeklinin bir tensörüdür, bunlar 32 görüntüye karşılık gelen etiketlerdir.

`image_batch` ve `labels_batch` tensörlerinde `.numpy()` öğesini çağırarak onları bir `numpy.ndarray`e dönüştürebilirsiniz.


## Performans için veri kümesini yapılandırın

I/O'nun bloke olmasına gerek kalmadan diskten veri alabilmeniz için arabelleğe alınmış önceden getirmeyi kullandığınızdan emin olalım. Bunlar, verileri yüklerken kullanmanız gereken iki önemli yöntemdir:

- `Dataset.cache`, ilk dönem boyunca diskten yüklendikten sonra görüntüleri bellekte tutar. Bu, modelinizi eğitirken veri kümesinin bir darboğaz haline gelmemesini sağlayacaktır. Veri kümeniz belleğe sığmayacak kadar büyükse, bu yöntemi, performanslı bir disk önbelleği oluşturmak için de kullanabilirsiniz.
- `Dataset.prefetch`, eğitim sırasında veri ön işleme ve model yürütme ile çakışıyor.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Verileri standartlaştırın

RGB kanal değerleri `[0, 255]` aralığındadır. Bu bir sinir ağı için ideal değildir; genel olarak girdi değerlerinizi küçük yapmaya çalışmalısınız.

Burada, `tf.keras.layers.Rescaling` kullanarak değerleri `[0, 1]` aralığında olacak şekilde standartlaştıracaksınız:

In [ ]:
normalization_layer = layers.Rescaling(1./255)

Bu katmanı kullanmanın iki yolu vardır. `Dataset.map` çağırarak veri kümesine uygulayabilirsiniz:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

Veya dağıtımı basitleştirebilecek katmanı model tanımınıza dahil edebilirsiniz. Burada ikinci yaklaşımı kullanalım.

Not: Görüntüleri daha önce `tf.keras.utils.image_dataset_from_directory` öğesinin `image_size` bağımsız değişkenini kullanarak yeniden boyutlandırmıştınız. 
Yeniden boyutlandırma mantığını modelinize de dahil etmek istiyorsanız, `tf.keras.layers.Resizing` katmanını kullanabilirsiniz.

# Modeli oluşturun

[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) model, her birinde maksimum ortaklama katmanı (max pooling layer) (`tf.keras.layers.MaxPooling2D`) bulunan üç evrişim bloğundan (`tf.keras.layers.Conv2D`) oluşur. Üstünde bir ReLU etkinleştirme işlevi (`'relu'`) tarafından etkinleştirilen 128 birimli tam bağlı bir katman (`tf.keras.layers.Dense`) vardır. Bu model yüksek doğruluk için ayarlanmamıştır—bu çalışma dosyasının amacı standart bir yaklaşım göstermektir.


In [ ]:

num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Modeli derleyin

Bu öğretici için, `tf.keras.optimizers.Adam` optimizer ve `tf.keras.losses.SparseCategoricalCrossentropy` kayıp(yitim) fonksiyonunu (loss function) seçin. Her eğitim dönemi için eğitim ve doğrulama kümesi başarımı görüntülemek için `metrics` bağımsız değişkenini `Model.compile` öğesine iletin.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Model özeti

Modelin `Model.summary` yöntemini kullanarak ağın tüm katmanlarını görüntüleyin:

In [ ]:
model.summary()

## Modeli eğitin

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Eğitim sonuçlarını görselleştirin

Eğitim ve doğrulama kümelerinin kayıp(yitim) (loss) ve doğruluk (accuracy) grafikleri oluşturun:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Grafikler, eğitim kümesi doğruluğunun ve doğrulama kümesi başarımı arasında büyük fark olduğunu ve modelin doğrulama kümesinde yalnızca yaklaşık %60 doğruluk elde ettiğini göstermektedir.

Neyin yanlış gittiğini inceleyelim ve modelin genel performansını artırmaya çalışalım.

## Overfitting (Aşırı uyum)

Yukarıdaki grafiklerde, eğitim kümesi doğruluğu zamanla doğrusal olarak artarken, doğrulama kümesi doğruluğu eğitim sürecinde yaklaşık %60 oranında durur. Ayrıca, eğitim ve doğrulama doğruluğu arasındaki doğruluk farkı dikkat çekicidir - [aşırım uyum(overfitting)](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) bir işareti.

Az sayıda eğitim örneği olduğunda, model bazen eğitim örneklerinden gürültülerden veya istenmeyen ayrıntılardan öğrenir - bir dereceye kadar bu, modelin yeni örnekler üzerindeki performansını olumsuz etkiler. Bu fenomen aşırı takma olarak bilinir. Bu, modelin yeni bir veri kümesi üzerinde genelleme yapmakta zorlanacağı anlamına gelir.

Eğitim sürecinde aşırı uyumla mücadele etmenin birçok yolu vardır. Bu çalışma dosyasında, *veri arttırmayı (data augmentation)* kullanacak ve modelinize *Dropout* ekleyeceksiniz.

## Veri Arttırma (Data augmentation)

Fazla takma genellikle az sayıda eğitim örneği olduğunda meydana gelir. Veri büyütme, inandırıcı görünen görüntüler veren rastgele dönüşümler kullanarak mevcut örneklerinizden ek eğitim verileri üretme yaklaşımını benimser. Bu, modeli verilerin daha fazla yönüne maruz bırakmaya ve daha iyi genelleştirmeye yardımcı olur.

Aşağıdaki Keras ön işleme katmanlarını kullanarak veri büyütmeyi uygulayacaksınız: `tf.keras.layers.RandomFlip`, `tf.keras.layers.RandomRotation` ve `tf.keras.layers.RandomZoom`. Bunlar, diğer katmanlar gibi modelinizin içine dahil edilebilir ve GPU üzerinde çalıştırılabilir.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

Aynı görüntüye birkaç kez veri büyütme uygulayarak birkaç artırılmış örneğin nasıl göründüğünü görselleştirelim:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(34):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

Bir modeli bir anda eğitmek için veri büyütmeyi kullanacaksınız.

## Seyreltme (Dropout)

Fazla takmayı azaltmak için başka bir teknik, ağa [seyreltme (dropout)](https://developers.google.com/machine-learning/glossary#dropout_regularization) düzenlemesini getirmektir.

Bir katmana bırakma uyguladığınızda, eğitim süreci sırasında katmandan bir dizi çıktı birimini rastgele (etkinleştirmeyi sıfıra ayarlayarak) bırakır. Dropout, girdi değeri olarak 0,1, 0,2, 0,4 vb. şeklinde bir kesirli sayı alır. Bu, uygulanan katmandan çıktı birimlerinin %10, %20 veya %40'ının rastgele bırakılması anlamına gelir.

Artırılmış görüntüleri kullanarak eğitmeden önce `tf.keras.layers.Dropout` ile yeni bir sinir ağı oluşturalım:

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Modeli derleyin ve eğitin

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Eğitim sonuçlarını görselleştirin

Veri büyütme ve `tf.keras.layers.Dropout` uygulandıktan sonra, eskisinden daha az fazla uyum olur ve eğitim ile doğrulama doğruluğu daha yakın hizalanır:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Yeni verilerle tahminde bulunun

Son olarak, eğitim veya doğrulama kümelerine dahil olmayan bir görüntüyü sınıflandırmak için modelimizi kullanalım.

Not: Veri büyütme ve bırakma katmanları, çıkarım zamanında etkin değildir.

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)